In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output
# from fbprophet import Prophet
from datetime import datetime,timedelta,date
import datetime
from clickhouse_driver import connect
import clickhouse_driver
import datetime as dList

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
#获取两个日期间的所有日期
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = dList.datetime.strptime(str(begin_date), "%Y-%m-%d")
    end_date = dList.datetime.strptime(str(end_date),"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += dList.timedelta(days=1)
    return date_list
#获取每月月末
def get_last_day(now_time):
    this_year = now_time.year
    if(now_time.month != 12):
        next_month = now_time.month + 1
    else:
        next_month = now_time.month
    return datetime.date(month=next_month, year=this_year, day=1)-timedelta(days=1)
# # 求出各个点的目标dau
# def standard_mau(this_date,this_month,last_day,target_mau):
#     print("this_date",this_date)
#     if len(this_date) != 0 :
#         k = (target_mau - 0) / (last_day.day - this_month.day)
#         b = 0
#         #如果是第一天则按照y=kx+b来算
#         date_lst = this_date[0][0].split('-')
#         this_date_time = datetime.date(year=int(date_lst[0]),month=int(date_lst[1]),day=int(date_lst[2]))
#         if this_date_time == this_month:
#             lst=[]
            
#             for i in range(0,len(this_date)):
                  
#                 date_list = this_date[i][0].split('-')
#                 lst.append((datetime.date(year=int(date_list[0]),month=int(date_list[1]),day=int(date_list[2])),int(k*int(date_list[2])+b)))
            
#             pd.DataFrame(lst, columns=['ds', 'MAU']).to_csv('target_mau.csv',index=False)
#         #如果不是第一天，则略过
#         else:
#             pass
#求预期的虚线
def standard_mau(this_start_month,this_end_month,this_month_target):
    delta = dList.datetime.strptime(str(this_end_month),"%Y-%m-%d").day - dList.datetime.strptime(str(this_start_month),"%Y-%m-%d").day +1
    k = this_month_target / delta
    b = -k
    this_all_date = getEveryDay(this_start_month,this_end_month)

    estList = []
    for i in range(0,delta+1):
        estX = i
        estY = round(k*(estX))
        if i!= 0:
            estList.append((dList.datetime.strptime(this_all_date[i-1],"%Y-%m-%d"),estY))
#     print(estList)
    pd.DataFrame(estList, columns=['ds', 'MAU']).to_csv('target_mau.csv',index=False)
# 获取数据
def get_data(date,j):
    # 获取clickhouse连接
    client = clickhouse_driver.Client(host='chi-ftabc-clickhouse-0-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2020')
    conn = connect(host='chi-ftabc-clickhouse-3-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2021')
    cursor = conn.cursor()
    # 获取当天和当月第一天
    cursor.execute('''select addDays(toDate('{}'),{}),toStartOfMonth(addDays(toDate('{}'),{}))'''.format(date,j,date,j))
    this_date,this_month = cursor.fetchall()[0]
    # 获取目标mau
    cursor.execute('''
    select count(distinct id) from dws_event_2020_middle_distribute 
    where toStartOfMonth(app_date)=toStartOfMonth(toDate('{}'))
    '''.format(str(this_month-timedelta(days=1))))
    this_month_target_mau = cursor.fetchall()[0][0]
    # 获取当前的mau
    cursor.execute('''
    select count(distinct id) from dws_event_2020_middle_distribute 
    where app_date between '{}' and '{}'
    '''.format(this_month,this_date))
    this_month_mau_till_now = int(cursor.fetchall()[0][0])
    this_month_mau_now = [(this_date,this_month_mau_till_now)]
    if this_date == this_month :
        pd.DataFrame(this_month_mau_now,columns=['ds','y']).to_csv('currrent_mau.csv',index=False)
    else :
        pd.DataFrame(this_month_mau_now,columns=['ds','y']).to_csv('currrent_mau.csv',index=False,mode='a',header=False)
     #获取本月最后一天
    last_day = get_last_day(this_month)
    #获取本月所有的天数
    cursor.execute('''
    select toString(app_date) from dws_event_2020_middle_distribute 
    where app_date>=('{}') and app_date<=toDate('{}') group by app_date order by app_date
    '''.format(this_date,last_day))
    days = int(str(this_date).split('-')[-1])
    all_day = cursor.fetchall()
    # 用第一天和最后一天使用y=kx+b得到本月每天应该达到的dau,每到月头重画一次
    standard_mau(this_month,last_day,this_month_target_mau)
    cursor.close()
    conn.close()
    if this_month_target_mau != 0:
        return this_month_mau_till_now,this_month_target_mau,round((this_month_mau_till_now/this_month_target_mau)*100,2)
    else :
        return this_month_mau_till_now,this_month_target_mau,0

In [8]:
i = 0
date='2020-03-01'
this_month_mau_till_now,this_month_target_mau,complete_rate_till_now = get_data(date,i)
this_month_mau = pd.read_csv('target_mau.csv')
all_dau_pdf=pd.read_csv('currrent_mau.csv')
all_dau_pdf.columns=['ds','y']
fig = go.Figure()
fig.add_trace(go.Scatter(x=this_month_mau['ds'], y=this_month_mau['MAU'], name='本月目标MAU',line=dict(color='firebrick', width=4,dash='dash')))
fig.add_trace(go.Scatter(x=all_dau_pdf['ds'], y=all_dau_pdf['y'], mode='lines',name='实际MAU',line=dict(color='royalblue', width=4)))

app.layout = html.Div(children=[
    dcc.Interval(id="counter", interval=1000, n_intervals=0),
    html.H1(children='MAU'),
    html.Div(id='goal',children='''至今：0 目标：500 完成率：0%'''),
    dcc.Graph(
        id='line',
        figure=fig
    ),
])
@app.callback(
    [Output(component_id='goal', component_property='children'),Output(component_id='line',component_property='figure')],
    [Input(component_id='counter', component_property='n_intervals'),Input('counter','n_intervals')]
)
def update_output_div(dropdown,n_intervals):
    global i
    i = i+1
    j=3
    this_month_mau_till_now,this_month_target_mau,complete_rate_till_now = get_data(date,i)
    all_dau_pdf=pd.read_csv('currrent_mau.csv')
    all_dau_pdf.columns=['ds','y']
    this_month_mau = pd.read_csv('target_mau.csv')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=this_month_mau['ds'], y=this_month_mau['MAU'], name='本月目标MAU',line=dict(color='firebrick', width=4, dash='dash')))
    fig.add_trace(go.Scatter(x=all_dau_pdf['ds'], y=all_dau_pdf['y'], mode='lines',name='实际MAU',line=dict(color='royalblue', width=4)))
    return '''至今：{} 目标：{} 完成率：{}%'''.format(this_month_mau_till_now,this_month_target_mau,complete_rate_till_now),fig

if __name__ == '__main__':
    app.run_server(debug=True,mode="inline")

In [7]:
#求预期的虚线
def estimate_retention(this_start_month,this_end_month,this_month_target):
    delta = datetime.strptime(this_end_month,"%Y-%m-%d").day - datetime.strptime(this_start_month,"%Y-%m-%d").day +1
    k = this_month_target / delta
    b = -k
    this_all_date = getEveryDay(this_start_month,this_end_month)

    estList = []
    for i in range(0,delta+1):
        estX = i
        estY = round(k*(estX))
        if i!= 0:
            estList.append((datetime.strptime(this_all_date[i-1],"%Y-%m-%d"),estY))
#     print(estList)
    pd.DataFrame(estList, columns=['ds', 'ret']).to_csv('target_ret.csv',index=False)

In [8]:
#获取两个日期间的所有日期
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = dList.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = dList.datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += dList.timedelta(days=1)
    return date_list

In [9]:
estimate_retention('2016-01-01','2016-01-31',5452)

AttributeError: module 'datetime' has no attribute 'strptime'